In [3]:
#필요한 모듈 import 

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data



In [5]:
# 1. Data Loading

mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [20]:
# 2. Model정의(Tensorflow graph 생성)

tf.reset_default_graph() #tensorflow graph초기화
# 2.1 placeholder

X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)

drop_rate = tf.placeholder(dtype=tf.float32)

#2.2 Convolution
## cNN은 이미지 학습에 최적화된 deep learning방법
##입력받은 이미지의 형태가 4차원 배열
##(이미지의 개수, 이미지의 width, 이미지의 height, color수)
X_img = tf.reshape(X, [-1,28,28,1])

# ## 2.3 Convolution Layer 1
# ## filter 정의 => 
# filter의 shape(width, height, color, filter수)
# filter1 = tf.Variable(tf.random_normal([3,3,1,32]))

# #filter를 이용해서 Convolution image를 생성
# L1 = tf.nn.conv2d(X_img, filter1, strides=[1,1,1,1], padding = "SAME")

# #만들어진 Convolution에 Relu를 적용
# L1 = tf.nn.relu(L1)
 
# ##pooling 작업(resize, sampling 작업) => optional
# #풀링작업하는이유 -> 필터적용해서 하다보면 이미지값이 계속 커짐.

# L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

#필터 만들고 convolution 하고 relu작업까지 한방에 가능~(실제구현)
L1 = tf.layers.conv2d(inputs=X_img, filters=32,
                     kernel_size=[3,3], padding="SAME",
                     strides=1,activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(inputs=L1,
                            pool_size=[2,2], 
                            padding="SAME",
                            strides=2)

#필터 만들고 convolution 하고 relu작업까지 한방에 가능~(실제구현)  << 2 >>
L2 = tf.layers.conv2d(inputs=L1, filters=64,
                     kernel_size=[3,3], padding="SAME",
                     strides=1,activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(inputs=L2,
                            pool_size=[2,2], 
                            padding="SAME",
                            strides=2)

print(L2.shape)
#(?, 7, 7, 64)
#2.3 Neural Network
#Convolution 의 결과를 Neural Network 의 입력으로 사용하기 위해 shape을 변경

L2 = tf.reshape(L2,[-1,7*7*64])

W1 = tf.get_variable("weight1", shape=[7*7*64,256],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name="bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W1)+b1)
layer1= tf.layers.dropout(_layer1, rate=drop_rate)


W2 = tf.get_variable("weight2", shape=[256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([10]), name="bias2")


#hypothesis
logits = tf.matmul(layer1,W2)+b2
H= tf.nn.relu(logits)


#cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

##train
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train=optimizer.minimize(cost)
# #
# #Session. 초기화
# sess=tf.Session()
# sess.run(tf.global_variables_initializer())

# #학습진행(batch처리)


(?, 7, 7, 64)


In [22]:

#Session. 초기화
sess=tf.Session()
sess.run(tf.global_variables_initializer())

#학습진행(batch처리)


#Accuracy 측정

In [24]:
##### 결국 우리 MNIST 예제는
##### 입력한 이미지 1개에 대해 예측한 결과가 
##### H 의 값으로 도출
####Hypothesis -> 10개의 값으로 이루어짐(위에꺼 참조하면)
##### [0.5 0.95 0.8 .0.34 0.12 0.34, ... ] 총 10개


#####앙상블은 이런 model이 여러개 있어요!
##즉, hypothesis 가 여러개 있어요
###  H1-> [0.5 0.95 0.8 .0.34 0.12 0.34, ... ] 총 10개
###  H2-> [0.2 0.3 0.4 .0.34 0.5 0.34, ... ] 총 10개
###  H3-> [0.2 0.4 0.4 .0.34 0.5 0.34, ... ] 총 10개
###  H4-> [0.1 0.5 0.7 .0.34 0.5 0.34, ... ] 총 10개




####SUM => [1.06, 1.43, 2.4 3.5.. ]총 10개
###최종 prediction은 SUM한 결과값을 가지고 예측
#이게바로 앙상블. Hypothesis 를 10개 해서 모델 10개를 구하고 나눠서 진짜
#최종 prediction 를구한다 -> 정확도 2%정도 올릴 수 있다.
